In [1]:
pip install intrinio_sdk

Note: you may need to restart the kernel to use updated packages.


In [2]:
import python_utils
import pandas as pd
import nltk
import numpy as np
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ehsan74\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
from __future__ import print_function
import time
import intrinio_sdk
from intrinio_sdk.rest import ApiException
from pprint import pprint

intrinio_sdk.ApiClient().configuration.api_key['api_key'] = 'OmRkZTJkOWRkYzBlNjg4MjUwYzIxZjE2ODgwYjRjYmNi'

company_api = intrinio_sdk.CompanyApi()

identifier = 'AAPL' # str | A Company identifier (Ticker, CIK, LEI, Intrinio ID)
page_size = 100 # int | The number of results to return (optional) (default to 100)
next_page = '' # str | Gets the next page of data from a previous API call (optional)

api_response_list = []
condition = True
i = 0
while condition:

    try:
        i = i+1
        api_response = company_api.get_company_news(identifier, page_size=page_size, next_page=next_page)
        api_response_list.append(api_response)
        next_page = api_response.next_page
        pprint('got page' + str(i))
#         time.sleep(63)
    except ApiException as e:
        print("Exception when calling CompanyApi->get_company_news: %s\r\n" % e)
        condition = False

'got page1'
'got page2'
'got page3'
'got page4'
'got page5'
'got page6'
'got page7'
'got page8'
'got page9'
'got page10'
'got page11'
'got page12'
'got page13'
'got page14'
'got page15'
'got page16'
'got page17'
'got page18'
'got page19'
'got page20'
'got page21'
'got page22'
'got page23'
'got page24'
'got page25'
'got page26'
'got page27'
'got page28'
'got page29'
'got page30'
'got page31'
'got page32'
'got page33'
'got page34'
'got page35'
'got page36'
'got page37'
'got page38'
'got page39'
'got page40'
'got page41'
'got page42'
'got page43'
'got page44'
'got page45'
'got page46'
'got page47'
'got page48'
'got page49'
'got page50'
'got page51'
'got page52'
'got page53'
'got page54'
'got page55'
'got page56'
'got page57'
'got page58'
'got page59'
'got page60'
'got page61'
'got page62'
'got page63'
'got page64'
'got page65'
'got page66'
'got page67'
'got page68'
'got page69'
'got page70'
'got page71'
'got page72'
'got page73'
'got page74'
'got page75'
'got page76'
'got page77'
'got pag

'got page594'
'got page595'
'got page596'
'got page597'
'got page598'
'got page599'
'got page600'
'got page601'
'got page602'
'got page603'
'got page604'
'got page605'
'got page606'
'got page607'
'got page608'
'got page609'
'got page610'
'got page611'
'got page612'
'got page613'
'got page614'
'got page615'
'got page616'
'got page617'
'got page618'
'got page619'
'got page620'
'got page621'
'got page622'
'got page623'
'got page624'
'got page625'
'got page626'
'got page627'
'got page628'
'got page629'
'got page630'
'got page631'
'got page632'
'got page633'
'got page634'
'got page635'
'got page636'
'got page637'
'got page638'
'got page639'
'got page640'
'got page641'
'got page642'
'got page643'
'got page644'
'got page645'
'got page646'
'got page647'
'got page648'
'got page649'
'got page650'
'got page651'
'got page652'
'got page653'
'got page654'
'got page655'
'got page656'
'got page657'
'got page658'
'got page659'
'got page660'
'got page661'
'got page662'
'got page663'
'got page664'
'got p

'got page1168'
'got page1169'
'got page1170'
'got page1171'
'got page1172'
'got page1173'
'got page1174'
'got page1175'
'got page1176'
'got page1177'
'got page1178'
'got page1179'
'got page1180'
'got page1181'
'got page1182'
'got page1183'
'got page1184'
'got page1185'
'got page1186'
'got page1187'
'got page1188'
'got page1189'
'got page1190'
'got page1191'
'got page1192'
'got page1193'
'got page1194'
'got page1195'
'got page1196'
'got page1197'
'got page1198'
'got page1199'
'got page1200'
'got page1201'
'got page1202'
'got page1203'
'got page1204'
'got page1205'
'got page1206'
'got page1207'
'got page1208'
'got page1209'
'got page1210'
'got page1211'
'got page1212'
'got page1213'
'got page1214'
'got page1215'
'got page1216'
'got page1217'
'got page1218'
'got page1219'
'got page1220'
'got page1221'
'got page1222'
'got page1223'
'got page1224'
'got page1225'
'got page1226'
'got page1227'
'got page1228'
'got page1229'
'got page1230'
'got page1231'
'got page1232'
'got page1233'
'got page1

'got page1715'
'got page1716'
'got page1717'
'got page1718'
'got page1719'
'got page1720'
'got page1721'
'got page1722'
'got page1723'
'got page1724'
'got page1725'
'got page1726'
'got page1727'
'got page1728'
'got page1729'
'got page1730'
'got page1731'
'got page1732'
'got page1733'
'got page1734'
'got page1735'
'got page1736'
'got page1737'
'got page1738'
'got page1739'
'got page1740'
'got page1741'
'got page1742'
'got page1743'
'got page1744'
'got page1745'
'got page1746'
'got page1747'
'got page1748'
'got page1749'
'got page1750'
'got page1751'
'got page1752'
'got page1753'
'got page1754'
'got page1755'
'got page1756'
'got page1757'
'got page1758'
'got page1759'
'got page1760'
'got page1761'
'got page1762'
'got page1763'
'got page1764'
'got page1765'
'got page1766'
'got page1767'
'got page1768'
'got page1769'
'got page1770'
'got page1771'
'got page1772'
'got page1773'
'got page1774'
'got page1775'
'got page1776'
'got page1777'
'got page1778'
'got page1779'
'got page1780'
'got page1

'got page2262'
'got page2263'
'got page2264'
'got page2265'
'got page2266'
'got page2267'
'got page2268'
'got page2269'
'got page2270'
'got page2271'
'got page2272'
'got page2273'
'got page2274'
'got page2275'
'got page2276'
'got page2277'
'got page2278'
'got page2279'
'got page2280'
'got page2281'
'got page2282'
'got page2283'
'got page2284'
'got page2285'
'got page2286'
'got page2287'
'got page2288'
'got page2289'
'got page2290'
'got page2291'
'got page2292'
'got page2293'
'got page2294'
'got page2295'
'got page2296'
'got page2297'
'got page2298'
'got page2299'
'got page2300'
'got page2301'
'got page2302'
'got page2303'
'got page2304'
'got page2305'
'got page2306'
'got page2307'
'got page2308'
'got page2309'
'got page2310'
'got page2311'
'got page2312'
'got page2313'
'got page2314'
'got page2315'
'got page2316'
'got page2317'
'got page2318'
'got page2319'
'got page2320'
'got page2321'
'got page2322'
'got page2323'
'got page2324'
'got page2325'
'got page2326'
'got page2327'
'got page2

In [11]:
#Extract URL, Title, Date and Summary of each piece of News
url_list = []
title_list = []
date_list = []
summary_list = []

cnt = 0
last = ''

for page_100 in api_response_list:
    for article in page_100.news:
        
        if last == article.title:
            cnt+= 1
            pass 
        last = article.title
        
        
        url_list.append(article.url)
        title_list.append(article.title)
        date_list.append(article.publication_date)
        summary_list.append(article.summary)

In [12]:
#Assigning news to dates within stock trading hours
#If news was released after 4pm (when the stock market closes), the news affects next day's stock price
from datetime import timedelta
new_date = []
for date_item in date_list:
    if date_item.hour < 15:
        new_date.append(date_item)
    elif date_item.hour > 16:
        new_date.append(date_item - timedelta(days=1))
    else:
        new_date.append(np.nan)

In [13]:
#Dataframe with each New's URL, Title, Date and Summary
new_df = pd.DataFrame ({'url':url_list,
                       'title':title_list,
                       'date':new_date,
                       'summary':summary_list})

new_df.to_csv('News DataFrame.csv')

In [14]:
#Read Apple's Stock data retrieved from Bloomberg
stock_data = pd.read_csv("Final Stock Data.csv", encoding = "ISO-8859-1")

In [15]:
new_df = pd.read_csv("News DataFrame.csv", encoding = "ISO-8859-1")

In [16]:
#Columns of interest 
stock_data = stock_data[['Dates','Date Close Price (n)', 'Next Day Close Price (n+1)', 'Difference in Price (n+1) - (n)', 'Percentage of Difference to Close Price', 'Label']]

In [17]:
#Match News data to Stock data
new_df['date']=pd.to_datetime(new_df['date']).dt.strftime('%Y-%m-%d')

In [18]:
stock_data['date']=pd.to_datetime(stock_data['Dates']).dt.strftime('%Y-%m-%d')

In [19]:
#Merging News and Stock data to one table
result = pd.merge(new_df, stock_data, on='date')
result

result.to_csv('Result 1.csv')